In [1]:
import torch
import torch.nn as nn
import pandas
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(10)
dtype = torch.FloatTensor

In [2]:
def generate_real():
    real_data = torch.FloatTensor(
            [np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2),
            np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2)])     # CPU上32位浮点型
    return real_data

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        # 初始化PyTorch父类
        super().__init__()

        # 定义神经网络层
        self.model = nn.Sequential(
                        nn.Linear(4, 3),
                        nn.Sigmoid(),
                        nn.Linear(3, 1),
                        nn.Sigmoid()
        )

        # 创建损失函数
        self.loss_function = nn.MSELoss()

        # 创建优化器, 使用随机梯度下降
        self.optimiser = torch.optim.SGD(self.parameters(), 
                                            lr=0.01)
        
    def forward(self,inputs):
        return self.model(inputs)
    
    def train(self, inputs, targets):
        # 计算网络的输出
        outputs = self.model(inputs)
        
        # 计算损失值
        loss = self.loss_function(outputs, targets)

        # 归零梯度，反向传播，并更新权重
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass
    


In [5]:
aa = Discriminator()
aa1 = generate_real()
print(aa1)
aa(aa1)


tensor([0.9543, 0.0042, 0.9267, 0.1498])


tensor([0.5866], grad_fn=<SigmoidBackward0>)